In [1]:
''' This code utilizes pyautogui to control the mouse instead of webscraping via selenium to more reliably avoid the 
captcha challenges and bot detection. This means that all button locations and coordinates of things in this code I had to 
manually find the coordinates of and this script only works when chrome (100% zoom) is opened with one tab (this code) and 
is maximized on a 1920x1080 monitor. Additionally since the loaction of the booking slot buttons changes as earlier slots 
become filled, this code is only reliable to book immediately once slots open up (the location of the buttons is referenced 
to the bottom of the screen where all the slots say 'not available yet'). The selenium code is better for missed booking 
slots and to check when they open up but is less reliable at not triggering the captcha. Ideally with this code you'd never 
miss a booking slot in the first place though and is much faster and simpler than the selenium alternative... This 
potentially takes more tweaking initially though but I can stay logged in to my booking account and I generally like this 
one more lol even though its less sophistocated and robust.
'''
# Import Libraries
import pyautogui as p # used to move the mouse and click and enter keystrokes
import time # used for wait function so clicks are not too fast
import random as r # used for randomizing click times and locations
from datetime import datetime as date # used to find the right spot on the calendar
from datetime import timedelta # used to run the code a week before the desired slot time right when bookings open
from twilio.rest import Client # this is a free service to be able to automatically text my phone when the code either books or fails
import pause # used to delay running the code till the booking slot actually opens

# Set Me!
day_to_book = 'Wednesday' # the day of the week to book - format like in the day_dict
date_to_book = '28' # the number date you want to book - format as a non-zero padded string (ex. '1', '2'...'10'etc)
time_to_book = '3 PM to  5 PM' # the booking slot time - format with single spaces between the hour and PM/AM but two spaces between the 'to' and the ending time
month = 4 # only need this for the pause function 
year = 2021 # only need this for the pause function

# Dictionaries Of The Locations Of Buttons On The Screen (could maybe do this relative to screen size for more robustness...)
calendar_dict_x = {'Monday':[493,520], 'Tuesday':[532,554], 'Wednesday':[566,593], 'Thursday':[605,629], 'Friday':[642,665], 'Saturday':[675,702], 'Sunday':[711,738]}
calendar_dict_y = [[800,817],[825,842],[851,867],[875,891],[900,915],[921,938]]
day_dict = {'Monday':1,'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6,'Sunday':7}
MTF_y = {'7 AM to  9 AM':[400,418], '8 AM to  10 AM':[437,458], '9 AM to  11 AM':[477,496], '10 AM to  12 PM':[516,535], '11 AM to  1 PM':[554,576],
                     '12 PM to  2 PM':[596,613], '1 PM to  3 PM':[635,651], '2 PM to  4 PM':[671,688], '3 PM to  5 PM':[713,728], '4 PM to  6 PM':[749,769],
                     '5 PM to  7 PM':[789,807], '6 PM to  8 PM':[829,846], '7 PM to  9 PM':[869,885], '8 PM to  10 PM':[907,928], '9 PM to  11 PM':[945,966]}
WTh_y = {'12 PM to  2 PM':[596,613], '1 PM to  3 PM':[635,651], '2 PM to  4 PM':[671,688], '3 PM to  5 PM':[713,728], '4 PM to  6 PM':[749,769],
                     '5 PM to  7 PM':[789,807], '6 PM to  8 PM':[829,846], '7 PM to  9 PM':[869,885], '8 PM to  10 PM':[907,928], '9 PM to  11 PM':[945,966]}
SS_y = {'10 AM to  12 PM':[554,576], '11 AM to  1 PM':[596,613], '12 PM to  2 PM':[635,651], '1 PM to  3 PM':[671,688], '2 PM to  4 PM':[713,728], 
                     '3 PM to  5 PM':[749,769], '4 PM to  6 PM':[789,807], '5 PM to  7 PM':[829,846], '6 PM to  8 PM':[869,885], '7 PM to  9 PM':[907,928], 
                     '8 PM to  10 PM':[945,966]}
slots_x = [899,951]

## Setup To Text Me Once Certain Events Occur
# the following line needs your Twilio Account SID and Auth Token
client = Client("ACa3f2f8227c2375850b8a8dd741b7d665", "81e2dedcf51b88ec9031e6e014e5e039")

# Finds The Time The Code Should Run (One Week Before The Desired Booking Slot Since That's When Booking Opens)
if time_to_book[2] == 'P':
    time_to_go = int(time_to_book[0]) + 12
elif time_to_book[2] == 'A':
    time_to_go = int(time_to_book[0])
elif time_to_book[3] == 'P': # the case where it is 12 PM ie Noon
    time_to_go = int(time_to_book[0:1])
elif time_to_book[3] == 'A':
    time_to_go = int(time_to_book[0:1])
else: 
    print('error in the date')
    client.messages.create(to = "+17789852567", from_ = "+12564004669", body = "Date Error")

year_before = (date(year, month, int(date_to_book)) - timedelta(days=7)).strftime('%Y')
month_before = (date(year, month, int(date_to_book)) - timedelta(days=7)).strftime('%m')
date_before = (date(year, month, int(date_to_book)) - timedelta(days=7)).strftime('%d')

# Determines Which Row Of The Calendar (treated like an array for indexing) Our Booking Day Is
ref_day = '01 ' + date.today().strftime('%m %Y')
day = date.strptime(ref_day, '%d %m %Y').weekday()
row = int((int(date_to_book) + int(day) - 1)/7)

In [2]:
## This Cell Executes All the Mouse Movements To Actually Book
# opens a second tab in this chrome window
new_tabx = r.randint(265,280)
new_taby = r.randint(10,24)
p.moveTo(new_tabx, new_taby, duration = 0.4-r.randint(0,3)/10)
p.click(new_tabx,new_taby)
time.sleep(0.5)

# enters the url to the hive booking website
url = 'https://app.rockgympro.com/b/widget/?a=list&&widget_guid=78c7ab47bbe04fa59fe0a4889f4cda4c&random=6075d2d653092&iframeid=&mode=p'
p.typewrite(url)
p.typewrite(["enter"])
time.sleep(1.5)

# waits until booking opens for the slot we want
pause.until(date(int(year_before), int(month_before), int(date_before), time_to_go))

# clicks on the info and bookings button
infox = r.randint(691,879)
infoy = r.randint(548,575)
p.moveTo(infox, infoy, duration = 0.4-r.randint(0,3)/10)
p.click(infox,infoy)
time.sleep(1)

# scrolls down a bit to where the members and calendar selection are
p.scroll(-200)
time.sleep(0.2)

# adds one member (can add clicks as the number of members increases but currently only can book one)
memberx = r.randint(879,900)
membery = r.randint(730,751)
p.moveTo(memberx, membery, duration = 0.4-r.randint(0,3)/10)
p.click(memberx,membery)
time.sleep(0.2)

# selects the day in the calendar based on the row and the column
dayx = r.randint(calendar_dict_x[day_to_book][0],calendar_dict_x[day_to_book][1])
dayy = r.randint(calendar_dict_y[row][0],calendar_dict_y[row][1])
p.moveTo(dayx, dayy, duration = 0.4-r.randint(0,3)/10)
p.click(dayx,dayy)
time.sleep(0.2)

# scroll all the way to the bottom of the page
p.scroll(-500)
time.sleep(0.3)

# selects the desired booking slot (number of available rows depends on the day since some days have less hours)
if day_to_book == 'Monday' or day_to_book == 'Tuesday' or day_to_book == 'Friday':
    slotx = r.randint(slots_x[0],slots_x[1])
    sloty = r.randint(MTF_y[time_to_book][0],MTF_y[time_to_book][1])
    p.moveTo(slotx, sloty, duration = 0.4-r.randint(0,3)/10)
    p.click(slotx,sloty)
    time.sleep(0.5)
elif day_to_book == 'Wednesday' or day_to_book == 'Thursday':
    slotx = r.randint(slots_x[0],slots_x[1])
    sloty = r.randint(WTh_y[time_to_book][0],WTh_y[time_to_book][1])
    p.moveTo(slotx, sloty, duration = 0.4-r.randint(0,3)/10)
    p.click(slotx,sloty)
    time.sleep(0.5)
elif day_to_book == 'Saturday' or day_to_book == 'Sunday':
    slotx = r.randint(slots_x[0],slots_x[1])
    sloty = r.randint(SS_y[time_to_book][0],SS_y[time_to_book][1])
    p.moveTo(slotx, sloty, duration = 0.4-r.randint(0,3)/10)
    p.click(slotx,sloty)
    time.sleep(0.5)
else: print('error')
    
# Uncomment this section if this part of the form is not already filled
#click my name to autofill
nextx = r.randint(512,563)
nexty = r.randint(579,598)
p.moveTo(nextx, nexty, duration = 0.4-r.randint(0,3)/10)
p.click(nextx,nexty)
time.sleep(0.3)

#click dropdown to confirm for location
nextx = r.randint(581,599)
nexty = r.randint(767,774)
p.moveTo(nextx, nexty, duration = 0.4-r.randint(0,3)/10)
p.click(nextx,nexty)
time.sleep(0.2)

#click YES to confirm for location
nextx = r.randint(496,596)
nexty = r.randint(819,833)
p.moveTo(nextx, nexty, duration = 0.4-r.randint(0,3)/10)
p.click(nextx,nexty)
time.sleep(0.2)

#click dropdown to confirm no covid symptoms
nextx = r.randint(575,599)
nexty = r.randint(959,968)
p.moveTo(nextx, nexty, duration = 0.4-r.randint(0,3)/10)
p.click(nextx,nexty)
time.sleep(0.2)

#click NO to confirm no covid symptoms
nextx = r.randint(495,579)
nexty = r.randint(1018,1031)
p.moveTo(nextx, nexty, duration = 0.4-r.randint(0,3)/10)
p.click(nextx,nexty)
time.sleep(0.2)

# time.sleep(0.2)
p.scroll(-r.randint(300,500))
# time.sleep(0.1)
# p.scroll(200,300)

#click continue
nextx = r.randint(587,706)
nexty = r.randint(944,968)
p.moveTo(nextx, nexty, duration = 0.4-r.randint(0,3)/10)
p.click(nextx,nexty)
time.sleep(0.8)

p.scroll(-r.randint(600,800))
time.sleep(0.2)
p.scroll(-r.randint(600,800))
time.sleep(0.2)
p.scroll(-r.randint(600,800))
time.sleep(0.2)

#click agree checkbox
nextx = r.randint(496,501)
nexty = r.randint(669,674)
p.moveTo(nextx, nexty, duration = 0.4-r.randint(0,3)/10)
p.click(nextx,nexty)
time.sleep(0.2)

#click captcha
nextx = r.randint(486,502)
nexty = r.randint(778,792)
p.moveTo(nextx, nexty, duration = 0.4-r.randint(0,3)/10)
p.click(nextx,nexty)
time.sleep(1)

#click complete booking
nextx = r.randint(595,718)
nexty = r.randint(860,873)
p.moveTo(nextx, nexty, duration = 0.4-r.randint(0,3)/10)
p.click(nextx,nexty)
time.sleep(0.3)
p.click(nextx,nexty)
time.sleep(0.3)
p.click(nextx,nexty)
time.sleep(0.3)

#change the "from_" number to your Twilio number and the "to" number to the phone number you signed up for Twilio with, or upgrade your account to send SMS to any phone number
client.messages.create(to = "+17789852567", from_ = "+12564004669", body = "Your Code Did A Thing and Needs Checking!")

<Twilio.Api.V2010.MessageInstance account_sid=ACa3f2f8227c2375850b8a8dd741b7d665 sid=SMce139551934e422194aec47c9f8402ab>